In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('bot_detection_data.csv')

In [ ]:
# Data Cleaning

import string
import nltk
# nltk.download() # to download any missing nltk packages
from nltk.corpus import stopwords
from nltk import word_tokenize

df.columns = df.columns.str.lower().str.replace(' ','_')

In [ ]:
def text_cleaning(a):
    remove_punctuation = [char for char in a if char not in string.punctuation]
    remove_punctuation = ''.join(remove_punctuation)
    return [word for word in remove_punctuation.split() if word.lower() not in stopwords.words('english')]

In [69]:
print(df.tweet.apply(text_cleaning))

KeyboardInterrupt: 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow_transformer = CountVectorizer(analyzer=text_cleaning).fit(df['tweet'])

bow_transformer.vocabulary_

In [ ]:
tweet_bow = bow_transformer.transform(df['tweet'])

print(tweet_bow)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(tweet_bow)
print(tfidf_transformer)

tweet_tfidf = tfidf_transformer.transform(tweet_bow)
print(tweet_tfidf)
print(tweet_tfidf.shape)

In [ ]:
X = tweet_tfidf
Y = df['bot_label']
from sklearn.naive_bayes import MultinomialNB

# Model training
model = MultinomialNB().fit(X,Y)

In [ ]:
# Predict Output
predicted = model.predict(X)

print("All Predictions:")
print(predicted)

In [ ]:
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score
)

y_pred = model.predict(X)
accuracy = accuracy_score(y_pred, Y)
f1 = f1_score(y_pred, Y, average="weighted")

print("Accuracy:", accuracy)
print("F1 Score:", f1)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

titles_options = [
    ("Confusion matrix, without normalization", None),
    ("Normalized confusion matrix", "true"),
]
for title, normalize in titles_options:
    disp = ConfusionMatrixDisplay.from_estimator(
        model,
        X,
        Y,
        cmap=plt.cm.Blues,
        normalize=normalize,
    )
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

plt.show()

In [ ]:
# Demo Section

sampleTweet = {'sample_tweet': ["Five Nights at Freddys"]}
sampleDF = pd.DataFrame(data=sampleTweet)
print(sampleDF)

In [ ]:
print(sampleDF.sample_tweet.apply(text_cleaning))

sample_tweet_bow = bow_transformer.transform(sampleDF['sample_tweet'])
# print(sample_tweet_bow)
sample_tfidf_transformer = TfidfTransformer().fit(sample_tweet_bow)
# print(sample_tfidf_transformer)
sample_tweet_tfidf = sample_tfidf_transformer.transform(sample_tweet_bow)
# print(sample_tweet_tfidf)
# print(sample_tweet_tfidf.shape)
X = sample_tweet_tfidf

predicted = model.predict(X)
print("Prediction:", predicted)
if predicted == 0:
    print("The tweet \"", sampleDF.loc[0].at['sample_tweet'], "\" does not come from bot",sep='')
else:
    print("The tweet \"", sampleDF.loc[0].at['sample_tweet'], "\" is from a bot account",sep='')